In [1]:
using OffsetArrays
using JuMP

In [2]:
MachinesParJob=[1 3 0; 1 2 3]
Duree=[6 7 0; 3 5 1]
bigM=sum(Duree)

22

In [49]:
function TestsSondabilite_relaxlin(tfin, faisable, varsbin, BestTfin, Bestsol,sondable)
    TA, TO, TR = false, false, false
    
    if(faisable == false)#Test de faisabilite
        TA=true
        println("TA")
    elseif(tfin >= BestTfin) #Test d'optimalite
        TO=true
        println("TO")
    #elseif( prod(abs.([round.(v, digits=0) for v in value.(varsbin)]-value.(varsbin)) .<= fill(10^-5, size(varsbin))) ) #Test de resolution
    elseif(sondable == false)
        TR=true
        println("TR")
        if (tfin <= BestTfin)
            Bestsol = value.(t)
            BestTfin=tfin
        end
    else
        println("non sondable")
    end
    TA, TO, TR, Bestsol, BestTfin
end

TestsSondabilite_relaxlin (generic function with 1 method)

In [50]:
function SeparerNoeud_relaxlin(varsshouldbebinary, listvars, listvals)
    # le noeud est non-sondable. Appliquer le critère de séparation pour le séparer en sous-noeuds 
    # et choisir un noeud-fils le plus à gauche   
    
    #find a fractionnal variable
    i, var = 1, 0.0
    while((i <= length(varsshouldbebinary)) && (var==0.0))
        #if(varsshouldbebinary[i] ∉ listvars)
        if(abs(round(value(varsshouldbebinary[i]), digits=0) - value(varsshouldbebinary[i]) ) >= 10^-5)
            var=varsshouldbebinary[i]

        end
        i+=1
    end
    
    #=
    #find most fractionnal variable
    i, var, maxfrac = -1, 0, 0.0
    for i in 1:length(varsshouldbebinary)
        if(abs(round(value(varsshouldbebinary[i]), digits=0) - value(varsshouldbebinary[i]) ) >= maxfrac) 
            #if a variable is more fractinonal
            var=varsshouldbebinary[i]
            maxfrac=abs(round(value(varsshouldbebinary[i]), digits=0) - value(varsshouldbebinary[i]) )
            #println(i, " ", var, " ", maxfrac)
        end
    end
    =#
    

    #set_lower_bound(var,1.0)
    #set_upper_bound(var,1.0)
    var = 1.0
    
    push!(listvars,var) #stocker l'identite de la variable choisie pour la séparation
    push!(listvals,1.0) #stocker la branche choisie, identifiee par la valeur de la variable choisie
    listvars, listvals
end

SeparerNoeud_relaxlin (generic function with 1 method)

In [51]:
function ExplorerAutreNoeud_relaxlin(listvars, listvals)
    #this node is sondable, go back to parent node then right child if possible
    
    stop=false
    #go back to parent node
    var=pop!(listvars)
    theval=pop!(listvals)
    #set_lower_bound(var,0.0)
    #set_upper_bound(var,1.0)
    var = 0.5

    #go to right child if possible, otherwise go back to parent
    while( (theval==0.0) && (length(listvars)>= 1))
        var=pop!(listvars)
        theval=pop!(listvals)
        #set_lower_bound(var,0.0) 
        #set_upper_bound(var,1.0)
        var = 0.5
    end
    if theval==1.0
        #set_lower_bound(var,0.0)
        #set_upper_bound(var,0.0)
        var = 0.0
        push!(listvars,var)
        push!(listvals,0.0)
    else
        println("\nFINISHED")
        stop=true
    end
    listvars, listvals, stop 
end

ExplorerAutreNoeud_relaxlin (generic function with 1 method)

In [52]:
#fonction d'affichage adaptable selon le problème
function Affichage(dis,V)
    println("  noeud   distance")
    
    for i in 0:V-1
        #print("\t\t%d" % (i, dis[i]))
        if i == 0
            println("  debut      ",dis[Int(i)])
        elseif i == 1
            println("  t1.1       ",dis[Int(i)])
        elseif i == 2
            println("  t2.1       ",dis[Int(i)])
        elseif i == 3
            println("  t1.2       ",dis[Int(i)])
        elseif i == 4
            println("  t2.2       ",dis[Int(i)])
        elseif i == 5
            println("  t2.3       ",dis[Int(i)])
        elseif i == 6
            println("  fin        ",dis[Int(i)])

        end
            
        
    end
end

Affichage (generic function with 1 method)

In [53]:
function BellmanFord(graph, V, E, src)
    
    faisable = true
    # Initialiser les distances avec la valeur infinie 
    dis = OffsetArray{Float64}(undef, 0:V-1)
    for i in 0:V-1
        
        dis[i] = -Inf
    end
        
    
    
  
    # initialiser la distance de la source en 0 
    dis[src] = 0
    
    #Mise à jour de la valeur des distances en suivant la méthode de Bellman Ford
    # d[v]=max(d[v], d[u] + w(u,v))
    for i in 0:V-2
        for j in 0:E-1
            if dis[graph[j][1]] + graph[j][3] > dis[graph[j][2]]
                dis[graph[j][2]] = dis[graph[j][1]] + graph[j][3]
            end
        end
    end
    
    
                            
    # cherche les cycles
    for i in 0:E-1                   
        x = graph[i][1] 
        y = graph[i][2] 
        weight = graph[i][3] 
        if dis[x] != -Inf && dis[x] + weight > dis[y]
                
            println("Le graphe se bloque dans un cycle")
            faisable = false
            return dis[x], faisable
            return
        end
    end
            
    Affichage(dis,V)
    return dis[V-1], faisable

end

BellmanFord (generic function with 1 method)

In [54]:
function ResolutionParGraphes(g,V,E)
    graph = OffsetArray{Array{Int64,1}}(undef, 0:E-1)
    #passer les valeurs du g qui commence par l'indice 1 à graph qui commence par l'indice 0.
    for i in 0:E-1
        graph[i] = g[i+1]
    end
    res,faisable=BellmanFord(graph, V, E, 0)
    return res, faisable
end
    

ResolutionParGraphes (generic function with 1 method)

In [55]:
listvars = []
listvals=[]

BestTfin=bigM
Bestsol=[]
V = 7 # Nombre de noeuds dans le graphe;
current_node_number=0
stop = false

x_2111 = 0.5
x_2312 = 0.5
sondable == false


#TA, TO, TR = false, false, false

varsshouldbebinary=[x_2111,x_2312]
while(!stop)
    
    println("\nNode number ", current_node_number, ": \n-----\n")

    print("Solve : start ... ")
    #status = optimize!(model2)
    #g = [[0, 1, 0], [0, 2, 0], [1, 3, 6],[3, 6, 7], [2, 4, 3],[4,5,5],[5,6,1],[2,1,3*x_2111],[1,2,6*(1-x_2111)],[4,3,1*x_2312],[3,4,7*(1-x_2312)]]
    println("x2111 = ", x_2111)
    if varsshouldbebinary[1] == 0.5 && varsshouldbebinary[2] == 0.5
        g = [[0, 1, 0], [0, 2, 0], [1, 3, 6],[3, 6, 7], [2, 4, 3],[4,5,5],[5,6,1]]
       
    elseif varsshouldbebinary[1] == 1.0 && varsshouldbebinary[2] == 1.0
        g = [[0, 1, 0], [0, 2, 0], [1, 3, 6],[3, 6, 7], [2, 4, 3],[4,5,5],[5,6,1],[2,1,3],[4,3,1]]
    elseif  varsshouldbebinary[1] == 0.0 && varsshouldbebinary[2] == 0.0
        g = [[0, 1, 0], [0, 2, 0], [1, 3, 6],[3, 6, 7], [2, 4, 3],[4,5,5],[5,6,1],[1,2,6],[3,4,7]]
    elseif  varsshouldbebinary[1] == 1.0 && varsshouldbebinary[2] == 0.5
        g = [[0, 1, 0], [0, 2, 0], [1, 3, 6],[3, 6, 7], [2, 4, 3],[4,5,5],[5,6,1],[2,1,3]]
    elseif  varsshouldbebinary[1] == 0.5 && varsshouldbebinary[2] == 1.0
        g = [[0, 1, 0], [0, 2, 0], [1, 3, 6],[3, 6, 7], [2, 4, 3],[4,5,5],[5,6,1],[4,3,1]]
    elseif  varsshouldbebinary[1] == 0.0 && varsshouldbebinary[2] == 0.5
        g = [[0, 1, 0], [0, 2, 0], [1, 3, 6],[3, 6, 7], [2, 4, 3],[4,5,5],[5,6,1],[1,2,6]]
    elseif  varsshouldbebinary[1] == 0.5 && varsshouldbebinary[2] == 0.0
        g = [[0, 1, 0], [0, 2, 0], [1, 3, 6],[3, 6, 7], [2, 4, 3],[4,5,5],[5,6,1],[3,4,7]]
    end
    E = length(g)
    tfin, faisable = ResolutionParGraphes(g,V,E)
    #println(tfin)
    
    println("... end")
    
    println("\nSolution precedemment memorisee ", Bestsol, " avec date de fin ", BestTfin, "\n")
    TA, TO, TR, Bestsol, BestTfin = TestsSondabilite_relaxlin(tfin, faisable, varsshouldbebinary, BestTfin, Bestsol,sondable)

    is_node_sondable = TA || TO || TR
    
    if(!is_node_sondable)
        #println("please")
        listvars, listvals = SeparerNoeud_relaxlin(varsshouldbebinary, listvars, listvals)
    else
        listvars, listvals, stop = ExplorerAutreNoeud_relaxlin(listvars, listvals)
    end
    println("listvals ",listvals)

    current_node_number = current_node_number + 1
end

println("\n******\n\nOptimal value = ", BestTfin, "\n\nOptimal t=", Bestsol)
   


Node number 0: 
-----

Solve : start ... x2111 = 0.5
  noeud   distance
  debut      0.0
  t1.1       0.0
  t2.1       0.0
  t1.2       6.0
  t2.2       3.0
  t2.3       8.0
  fin        13.0
... end

Solution precedemment memorisee Any[] avec date de fin 22

non sondable
listvals Any[1.0]

Node number 1: 
-----

Solve : start ... x2111 = 0.5
  noeud   distance
  debut      0.0
  t1.1       0.0
  t2.1       0.0
  t1.2       6.0
  t2.2       3.0
  t2.3       8.0
  fin        13.0
... end

Solution precedemment memorisee Any[] avec date de fin 22

non sondable
listvals Any[1.0, 1.0]

Node number 2: 
-----

Solve : start ... x2111 = 0.5
  noeud   distance
  debut      0.0
  t1.1       0.0
  t2.1       0.0
  t1.2       6.0
  t2.2       3.0
  t2.3       8.0
  fin        13.0
... end

Solution precedemment memorisee Any[] avec date de fin 22

non sondable
listvals Any[1.0, 1.0, 1.0]

Node number 3: 
-----

Solve : start ... x2111 = 0.5
  noeud   distance
  debut      0.0
  t1.1       0.0
  

LoadError: InterruptException:

In [158]:
println("\n******\n\nOptimal value = ", BestTfin, "\n\nOptimal t=", Bestsol)


******

Optimal value = 22

Optimal t=Any[]
